<a href="https://colab.research.google.com/github/sindla97/RAG/blob/dev/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aproach
1. import pdf files to the knowledge base
2. Perfrom semantic segmentation using a sentense transfromer type model and convert them to embeddings
3. use a rerank model to rank the retrived embeddings
4. Use the LLM to generate response based on the query and retrvied emebeddings


Try to provide an option to update the knowlege base when new documents are provided,

In [2]:
!pip install langchain-google-genai pypdf langchain_experimental langchain-pinecone langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.6/441.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.0/368.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/

In [2]:
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "/content/AI_Agent_white_paper_by_google_1737132048.pdf"
loader = PyPDFLoader(pdf_path)
loader2=PyPDFLoader(pdf_path, mode='single')

In [3]:
all_pages=loader.load()
# we can use load_and_split() but we have to provide the textsplitter criterion if not it uses Recrusivetextsplitter by default
all_pages[10].page_content

'Agents\n11\nSeptember 2024\nd. Action input: The model’s decision on what inputs to provide to the tool (if any)\ne. Observation: The result of the action / action input sequence\ni. This thought / action / action input / observation could repeat N-times as needed\nf. Final answer: The model’s final answer to provide to the original user query\n4. The ReAct loop concludes and a final answer is provided back to the user\nFigure 2. Example agent with ReAct reasoning in the orchestration layer\nAs shown in Figure 2, the model, tools, and agent configuration work together to provide \na grounded, concise response back to the user based on the user’s original query. While \nthe model could have guessed at an answer (hallucinated) based on its prior knowledge, \nit instead used a tool (Flights) to search for real-time external information. This additional \ninformation was provided to the model, allowing it to make a more informed decision based \non real factual data and to summarize this 

### loading the whole document and invoking the llm for summary and generating embeddings would work for small documents but when the documnet size is large, it has to be chunked and summarized indiviudally and combine to generating embeddings


#Summarize using the whole document

In [4]:
whole_doc=loader2.load() # loading the whole pdf as a single document for answering summary type questions
whole_doc[0].page_content

'Agents\nAuthors: Julia Wiesinger, Patrick Marlow  \nand Vladimir Vuskovic\n\x0cAgents\n2\nSeptember 2024\nAcknowledgements\nReviewers and Contributors\nEvan Huang\nEmily Xue\nOlcan Sercinoglu\nSebastian Riedel\nSatinder Baveja\nAntonio Gulli\nAnant Nawalgaria\nCurators and Editors\nAntonio Gulli\nAnant Nawalgaria\nGrace Mollison \nTechnical Writer\nJoey Haymaker\nDesigner\nMichael Lanning\n\x0cIntroduction 4\nWhat is an agent? 5\n The model 6\n The tools 7\n The orchestration layer 7\n Agents vs. models 8\n Cognitive architectures: How agents operate  8\nTools: Our keys to the outside world 12\n Extensions  13\n  Sample Extensions  15\n Functions  18\n  Use cases 21\n  Function sample code 24\n Data stores 27\n  Implementation and application 28\n Tools recap 32\nEnhancing model performance with targeted learning 33\nAgent quick start with LangChain 35\nProduction applications with Vertex AI agents 38\nSummary 40\nEndnotes 42\nTable of contents\n\x0cAgents\n4\nSeptember 2024\nIntroduc

In [5]:
from google.colab import userdata
google_api_key=userdata.get('GOOGLE_API_KEY')


In [6]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings

# Initialize the text splitter
text_splitter = SemanticChunker(GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key))
chunks = text_splitter.create_documents([''.join([page.page_content for page in all_pages])]) # create document takes in list of dcoumetns and splits it, we have to pass the whole document as string

In [6]:
chunks[0].page_content

'Agents\nAuthors: Julia Wiesinger, Patrick Marlow  \nand Vladimir VuskovicAgents\n2\nSeptember 2024\nAcknowledgements\nReviewers and Contributors\nEvan Huang\nEmily Xue\nOlcan Sercinoglu\nSebastian Riedel\nSatinder Baveja\nAntonio Gulli\nAnant Nawalgaria\nCurators and Editors\nAntonio Gulli\nAnant Nawalgaria\nGrace Mollison \nTechnical Writer\nJoey Haymaker\nDesigner\nMichael LanningIntroduction 4\nWhat is an agent? 5\n The model 6\n The tools 7\n The orchestration layer 7\n Agents vs. models 8\n Cognitive architectures: How agents operate  8\nTools: Our keys to the outside world 12\n Extensions  13\n  Sample Extensions  15\n Functions  18\n  Use cases 21\n  Function sample code 24\n Data stores 27\n  Implementation and application 28\n Tools recap 32\nEnhancing model performance with targeted learning 33\nAgent quick start with LangChain 35\nProduction applications with Vertex AI agents 38\nSummary 40\nEndnotes 42\nTable of contentsAgents\n4\nSeptember 2024\nIntroduction\nHumans are f

In [7]:
len(chunks)

18

In [7]:
# importing the llm to summarize the whole document  an retrvial
#send the retrived documents to llm and ouptut in a structured format
from langchain_google_genai import ChatGoogleGenerativeAI

output_llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash", google_api_key=google_api_key)

#lets build a message


In [9]:
messages = [
    ("system", "please provide a summary of this document"),
    ("human", f"please proivde a detailed summary of the document: {whole_doc[0].page_content}"),
]
summary=output_llm.invoke(messages)
summary # summary of the whole document

AIMessage(content='This whitepaper, "Agents," by Wiesinger, Marlow, and Vuskovic, explores Generative AI agents – applications that achieve goals by observing and acting upon the world using available tools.  The authors differentiate agents from models, highlighting agents\' ability to interact with external systems, manage session history, and employ native logic layers via cognitive architectures.\n\nThe paper details the three core components of an agent\'s architecture:\n\n1. **The Model:**  A language model (LM) acting as the central decision-maker, utilizing reasoning frameworks like ReAct, Chain-of-Thought, or Tree-of-Thoughts.  The model isn\'t typically trained with the agent\'s specific configuration but can be refined with examples of the agent\'s tool usage.\n\n2. **The Tools:**  These bridge the gap between the model and the external world, enabling interaction with data and services.  The paper focuses on three types:\n    * **Extensions:** Standardized ways to connect a

In [10]:
from langchain.schema import Document

ai_message = summary.content
doc = Document(page_content=ai_message) # convert the AI message output to a documemnt

print(doc)

page_content='This whitepaper, "Agents," by Wiesinger, Marlow, and Vuskovic, explores Generative AI agents – applications that achieve goals by observing and acting upon the world using available tools.  The authors differentiate agents from models, highlighting agents' ability to interact with external systems, manage session history, and employ native logic layers via cognitive architectures.

The paper details the three core components of an agent's architecture:

1. **The Model:**  A language model (LM) acting as the central decision-maker, utilizing reasoning frameworks like ReAct, Chain-of-Thought, or Tree-of-Thoughts.  The model isn't typically trained with the agent's specific configuration but can be refined with examples of the agent's tool usage.

2. **The Tools:**  These bridge the gap between the model and the external world, enabling interaction with data and services.  The paper focuses on three types:
    * **Extensions:** Standardized ways to connect agents to APIs, us

In [11]:
chunks.append(doc) # append the document tothe other chunks

In [8]:
# lets use a vector store  to store these embeddings
from langchain_pinecone.vectorstores import PineconeVectorStore
vecstore=PineconeVectorStore(embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key),pinecone_api_key=userdata.get('pinecone_api'), index_name='rag-test')
vecstore

In [ ]:
vecstore.delete(delete_all=True) # delete any existing embeddings in the index
vecstore.add_documents(chunks) # create embeddings (based on chunk's text) and add to Pinecone

In [13]:
vecstore.search(query='what are the contents of the document Agentic AI', search_type='similarity') # quick check for the retrival

[Document(id='ba7d7da5-529b-4abc-992c-30128de17ec2', metadata={}, page_content="6. Yao, S. et al., 2023, 'Tree of Thoughts: Deliberate Problem Solving with Large Language Models'. Available at:  \nhttps://arxiv.org/abs/2305.10601 . 7. Long, X., 2023, 'Large Language Model Guided Tree-of-Thought'. Available at:  \nhttps://arxiv.org/abs/2305.08291 ."),
 Document(id='2d1eaef7-0b5c-41bc-9e08-68ea15a8a12e', metadata={}, page_content="Diao, S. et al., 2023, 'Active Prompting with Chain-of-Thought for Large Language Models'. Available at:  \nhttps://arxiv.org/pdf/2302.12246.pdf ."),
 Document(id='ca04dd70-907a-4f7d-a89f-0dc25546aaf0', metadata={}, page_content='Agents\nAuthors: Julia Wiesinger, Patrick Marlow  \nand Vladimir VuskovicAgents\n2\nSeptember 2024\nAcknowledgements\nReviewers and Contributors\nEvan Huang\nEmily Xue\nOlcan Sercinoglu\nSebastian Riedel\nSatinder Baveja\nAntonio Gulli\nAnant Nawalgaria\nCurators and Editors\nAntonio Gulli\nAnant Nawalgaria\nGrace Mollison \nTechnical 

In [14]:
vecstore.search(query='what is the summary of the document Agentic AI', search_type='similarity', k=4)

[Document(id='ba7d7da5-529b-4abc-992c-30128de17ec2', metadata={}, page_content="6. Yao, S. et al., 2023, 'Tree of Thoughts: Deliberate Problem Solving with Large Language Models'. Available at:  \nhttps://arxiv.org/abs/2305.10601 . 7. Long, X., 2023, 'Large Language Model Guided Tree-of-Thought'. Available at:  \nhttps://arxiv.org/abs/2305.08291 ."),
 Document(id='33d19396-5a71-43eb-a337-81a66e201841', metadata={}, page_content='3.'),
 Document(id='2d1eaef7-0b5c-41bc-9e08-68ea15a8a12e', metadata={}, page_content="Diao, S. et al., 2023, 'Active Prompting with Chain-of-Thought for Large Language Models'. Available at:  \nhttps://arxiv.org/pdf/2302.12246.pdf ."),
 Document(id='59d6cc00-e59d-446b-9cd5-9e5ccba721fd', metadata={}, page_content='4.')]

In [15]:
user_query='what is the summary of the document Agents'
retrived_contents=vecstore.search(query=user_query, search_type='similarity', k=4)
message=f"Based on the contents provided {''.join([x.page_content for x in retrived_contents])} answer the user question:{user_query} in a structured format and ask a followup question"
print(output_llm.invoke(message).content)

**Document: Agents**

**Summary:**

This document, authored by Julia Wiesinger, Patrick Marlow, and Vladimir Vuskovic, explores the concept of agents in the context of artificial intelligence.  It defines agents, detailing their components:  models, tools (including extensions and functions), and an orchestration layer.  The document contrasts agents with models, explaining how cognitive architectures govern agent operation.  It provides a detailed overview of tools, focusing on extensions, functions, and data stores, including sample code and use cases.  The document also discusses enhancing model performance through targeted learning and offers quick starts using LangChain and Vertex AI agents for production applications.  Finally, it includes acknowledgements to various reviewers, contributors, curators, editors, the technical writer, and designer.


**Structured Format:**

| Section             | Description                                                                        |
|

In [ ]:
def chat(user_query):
  retrived_contents=vecstore.search(query=user_query, search_type='similarity', k=4)
  message=f"Based on the contents provided {''.join([x.page_content for x in retrived_contents])} answer the user question:{user_query} in a structured format and ask a followup question"
  print(output_llm.invoke(message).content)
  new_user_query=input()
  if len(new_user_query)>0:
    chat(new_user_query)

chat('what are the contents of the document Agentic AI')

# lets summarize the whole document and add it to the vector store
we can utilize langchain summarize feature but it requires frequent invokes with the llm, if the llm is hosted and accessed through inference endpoints it does cost and trigger alerts for quota and usage


## To access models in huggingface we have2 options
### 1. using HuggingFaceEndpoint
### 2.HuggingFacePipeline
## huggingfaceendpoint helps hosting the model in remote server and takes care of the architecure but we do have to pay as a service
## huggingfacepipeline downloads the model into your local and you would need GPU to host and run the model but it is free of cost


In [9]:
userdata.get('huggingface_api')
from huggingface_hub import login
login(token=userdata.get('huggingface_api'))

In [11]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFacePipeline
from transformers import pipeline
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


hf = HuggingFacePipeline.from_model_id(
    model_id="google/gemma-3-1b-it",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 20000, 'trust_remote_code':True
     }
)
#HuggingFacePipeline will download model weights and host model locally
# i did not have enough memory to summary the document,
# one can try HuggingFaceEndpoint to use inference providers for the summary but is a paid service

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Device set to use cpu


In [12]:
from langchain.chains import load_summarize_chain
summarizer=load_summarize_chain(llm=hf, chain_type='map_reduce')
summarizer

MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7e0632442990>, model_id='google/gemma-3-1b-it', model_kwargs={'torch_dtype': 'auto'}, pipeline_kwargs={'max_new_tokens': 20000, 'trust_remote_code': True}), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0

In [ ]:
summary=summarizer.invoke(chunks)
summary
